In [16]:
!pip install -q agno nest-asyncio asyncio openai

In [17]:
import asyncio
import random
import re
import os
import nest_asyncio
import logging

from agno.agent import Agent
from agno.models.openai import OpenAIChat
from agno.guardrails import BaseGuardrail
from agno.exceptions import InputCheckError, CheckTrigger
from agno.run.agent import RunInput

In [18]:
logging.getLogger("agno").setLevel(logging.CRITICAL)

api_key = "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"
base_url = "https://openrouter.ai/api/v1"

os.environ["OPENAI_API_KEY"] = api_key
os.environ["OPENAI_BASE_URL"] = base_url

BANNED_PATTERNS = [
    r"\b(дурак|кретин|идиот)\b",
    r"\b(ненавижу|убью|расстрелять)\b",
]

TOPIC_KEYWORDS = ["lada niva", "лада нива", "niva", "ваз-2121"]

def is_offtopic(text: str) -> bool:
    lowered = text.lower()
    return not any(kw in lowered for kw in TOPIC_KEYWORDS)

def has_toxicity(text: str) -> bool:
    lowered = text.lower()
    return any(re.search(pattern, lowered) for pattern in BANNED_PATTERNS)

class ConversationGuardrail(BaseGuardrail):
    def check(self, run_input: RunInput) -> None:
        content = run_input.input_content
        if not isinstance(content, str):
            return
        if has_toxicity(content):
            raise InputCheckError(
                "Давай оставаться в рамках уважительной беседы — без оскорблений.",
                check_trigger=CheckTrigger.INPUT_NOT_ALLOWED,
            )
        if is_offtopic(content):
            raise InputCheckError(
                "Сфокусируемся на теме Lada Niva и её феномене. Переформулируй мысль ближе к теме.",
                check_trigger=CheckTrigger.INPUT_NOT_ALLOWED,
            )

    async def async_check(self, run_input: RunInput) -> None:
        return self.check(run_input)

guardrail = ConversationGuardrail()

model = OpenAIChat(
    id="Qwen/Qwen3-Next-80B-A3B-Instruct",
    api_key=api_key,
    base_url=base_url,
    max_tokens=500,
    temperature=0.8,
)

In [19]:
panelists = [
    Agent(
        model=model,
        name="Артём Инженер",
        instructions=(
            "Ты — Артём, автомобильный инженер и техноблогер. "
            "Твой стиль: спокойный, структурированный, любишь цифры и теххарактеристики. "
            "Объясняешь конструкцию, надёжность, обслуживание. "
            "Пример: объясни, почему Lada Niva стала культовой с точки зрения инженерии."
        ),
        markdown=False,
        pre_hooks=[guardrail],
    ),
    Agent(
        model=model,
        name="Лена Автожурналист",
        instructions=(
            "Ты — Лена, автожурналист и ведущая подкаста про историю моделей. "
            "Стиль: образный, но информативный. "
            "Говоришь о контексте, восприятии и рынке. "
            "Пример: Lada Niva — символ бездорожья и простоты, понятный каждому."
        ),
        markdown=False,
        pre_hooks=[guardrail],
    ),
    Agent(
        model=model,
        name="Макс Дрифт",
        instructions=(
            "Ты — Макс, энтузиаст тюнинга. "
            "Стиль: эмоциональный, яркий, с примерами из форумов. "
            "Говоришь о доработках, экспедициях и тюнинге Lada Niva."
        ),
        markdown=False,
        pre_hooks=[guardrail],
    ),
    Agent(
        model=model,
        name="Игорь Мемолог",
        instructions=(
            "Ты — Игорь, мемолог с иронией, но без оскорблений. "
            "Сравниваешь Lada Niva с героями фильмов и мемами, подчёркивая её живучесть."
        ),
        markdown=False,
        pre_hooks=[guardrail],
    ),
]


In [20]:
async def relay_message(sender: Agent, receiver: Agent, message: str, step: int):
    print(f"\nШаг {step}: {sender.name} → {receiver.name}")
    print(f"Сообщение:\n{message}\n")
    try:
        answer = receiver.run(message)
        text = answer.content if hasattr(answer, "content") else str(answer)
    except InputCheckError as e:
        text = e.message
    except Exception as e:
        text = f"Техническая ошибка при ответе модели: {type(e).__name__}: {e}"
    print(f"{receiver.name} отвечает:\n{text}\n")
    print("-" * 70)
    return text

In [21]:
async def run_panel_random(rounds: int = 4):
    print("КРУГЛЫЙ СТОЛ: Lada Niva как феномен бездорожья")
    topic_intro = (
        "Предлагаю обсудить Lada Niva. "
        "Почему эта модель стала устойчивым символом бездорожья и экспедиций?"
    )
    current_speaker = random.choice(panelists)
    print(f"\nНачинает {current_speaker.name}:\n")
    try:
        first_reply = current_speaker.run(topic_intro)
        current_text = first_reply.content if hasattr(first_reply, "content") else str(first_reply)
    except Exception as e:
        print(f"Техническая ошибка при старте диалога: {type(e).__name__}: {e}")
        return
    print(current_text)
    print("-" * 70)
    for i in range(1, rounds):
        next_candidates = [p for p in panelists if p is not current_speaker]
        next_speaker = random.choice(next_candidates)
        current_text = await relay_message(current_speaker, next_speaker, current_text, step=i + 1)
        current_speaker = next_speaker
        await asyncio.sleep(0.5)

In [22]:
async def run_panel_sequential():
    print("=" * 70)
    print("ПОСЛЕДОВАТЕЛЬНОЕ ОБСУЖДЕНИЕ: взгляд по кругу")
    print("Тема: Lada Niva и её культовый статус")
    print("=" * 70)
    topic_intro = (
        "Давайте по очереди выскажемся: в чём феномен Lada Niva и почему она стала культовой?"
    )
    order = [0, 1, 2, 3]
    current_text = topic_intro
    for step_index in range(len(order) - 1):
        sender = panelists[order[step_index]]
        receiver = panelists[order[step_index + 1]]
        current_text = await relay_message(sender, receiver, current_text, step=step_index + 1)
        await asyncio.sleep(0.5)

In [23]:
async def main():
    await run_panel_random(rounds=4)
    print("\n\nТеперь тот же состав, но в фиксированном порядке:\n")
    await run_panel_sequential()

if __name__ == "__main__":
    nest_asyncio.apply()
    asyncio.run(main())

КРУГЛЫЙ СТОЛ: Lada Niva как феномен бездорожья

Начинает Артём Инженер:

Конечно. Lada Niva — не просто «бабушкина машинка» или «дешёвый кроссовер». Это **инженерный шедевр советского периода**, который, несмотря на устаревший дизайн, остался непревзойдённым в своём сегменте — и вот почему.

---

### 🔧 **1. Постоянный полный привод — без компромиссов**
Большинство внедорожников 80–90-х (например, Jeep Cherokee или Toyota Land Cruiser) имели **переключаемый полный привод** — включался вручную, только на бездорожье.  
Niva — **постоянный 4x4** с **механической блокировкой межосевого дифференциала**. То есть:  
— 4 колеса всегда вращаются с одинаковой частотой,  
— нет «сцепления» между осями, как у современных систем с электроникой,  
— **никаких сбоев от влаги, грязи, мороза — работает при -40°C и глубокой колее**.

> ✅ *Инженерный приём: 2-ступенчатая раздатка с редуктором (2,08:1)* — даёт **перегрузку в 2 раза**, позволяя преодолевать **45-градусные склоны** и **глубокий снег до 60 см